In [17]:
import pandas as pd
import numpy as np
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

from visual_dialog import dialog_utils

In [38]:
num_examples = 1
num_qa = 5
# ret_img = True
include_Q_A = False
gpt_prompt = False

In [39]:
dialog_annotations_folder = './visual_dialog/annotations'

# experiment_name = f'{num_examples}_examples_{num_qa}_qa_{"ret_img" if ret_img else "ret_text"}_{"gpt_prompt" if gpt_prompt else "nocontext"}'
experiment_name = f'{num_examples}_examples_{num_qa}_qa_{"inc_QA" if include_Q_A else "only_caption"}{"_GPT" if gpt_prompt else ""}'

baseline_experiment = f'{num_examples}_examples_{num_qa}_qa_{"inc_QA" if include_Q_A else "only_caption"}'

dialog_annotations_csv_path = f'{dialog_annotations_folder}/{experiment_name}.csv'

if os.path.exists(dialog_annotations_csv_path):
    annotations_df = pd.read_csv(dialog_annotations_csv_path, encoding='utf8')
else:
    if not os.path.exists(dialog_annotations_folder):
        os.makedirs(dialog_annotations_folder)
    annotations_df = pd.DataFrame({ 
        'id': pd.Series(dtype='int'), 'prediction': pd.Series(dtype='int') })

In [40]:
# scan experiment folder without recursion
data_path = f'./visual_dialog/{experiment_name}.json'
baseline_path = f'./visual_dialog/{baseline_experiment}.json'
baseline_prompts = None

with open(data_path, 'r') as file:
    data = json.load(file)

if data_path != baseline_path:
    with open(baseline_path, 'r') as file:
        baseline_data = json.load(file)
    baseline_prompts = baseline_data['prompts']

# Retrieve the lists
prompts = data['prompts']
outputs = data['outputs']

print(f"Amount of samples:", len(prompts), "\n")

for i in range(len(prompts)):
    # skip annotated story
    if i in annotations_df['id'].values:
        continue
    # if baseline_prompts != None:
    if baseline_prompts:
        print("Unaugmented text input:")
        dialog_utils.display_prompt(baseline_prompts[i])
        print("\nGPT augmented text input\n")
    
    dialog_utils.display_prompt(prompts[i])
    print('\n', '=' * 50, '\n')
    dialog_utils.display_output(outputs[i])

    prediction = ''
    while (prediction != '0') and (prediction != '0.5') and (prediction != '1') and (prediction != '2'):
        prediction = input('prediction: 0 (false) - 0.5 (partly correct) - 1 (correct) - 2 (stop)')
    
    if prediction == '2':
        clear_output(wait=False)
        break

    annotations_df.loc[len(annotations_df)] = [i, float(prediction)]

    clear_output(wait=False)

annotations_df.to_csv(dialog_annotations_csv_path, encoding='utf8', index=False)

print('\n', 'Done', '\n')

Amount of samples: 50 


 Done 



In [42]:
# Calculate accuracy
print(dialog_annotations_csv_path)
annotations_df = pd.read_csv(dialog_annotations_csv_path)
accuracy = round(sum(annotations_df['prediction']) / len(annotations_df['prediction']) * 100, 2)
print(f'The accuracy of 50 captions {"with questions and answers" if include_Q_A else ""}{"with adjusted prompt by GPT" if gpt_prompt else ""}is {accuracy}')

./visual_dialog/annotations/1_examples_5_qa_only_caption.csv
The accuracy of 50 captions is 54.0
